In [ ]:
%matplotlib inline
import os
import numpy as np
import sys
import matplotlib
import matplotlib.pyplot as plt
c=3.e8 #speed of light

filepath='D:/measuring/'
import scipy.signal
os.chdir(filepath)

sys.path.append('H:/My Documents/measuring/')

data_dir='D:\measuring\data/20160503\ON diamond\pos 7/raw data spectrometer\L7'#'D:\measuring\data/20160707\CL2_newmirror_processed'#"/Users/suzannevandam/Documents/PhD/localdata/20160430/ON_diamond" 
print data_dir

%matplotlib inline

In [ ]:
import analysis.scripts.cavity.spectrometer_analysis as sa
import analysis.scripts.cavity.spectrometer_analysis_2Dsweep as sa_2D

# Finding the cavity length from a 2D plot on diamond

1. load the data from data_dir containing your 2D sweep data, and check if it looks fine

In [ ]:
#note that what are called wavelengths, are actually the frequencies in THz! 
wavelengths,filenumbers,intensities = sa.load_data_from_folder(data_dir)

In [ ]:
plt.plot(wavelengths,intensities[:,0])

In [ ]:
reload(sa_2D)
sa_2D.plot_data(data_dir,wavelengths,intensities,vmax=1000, aspect =0.1)

2. Find the peaks of one of the spectrometer sweep, and check if your settings (especially: minimum_peak_height) is fine.
   check with 2D data if it works well 

In [ ]:
reload(sa_2D)
minimum_peak_height = 500
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,0],plot_peak_locations=True,minimum_peak_height=minimum_peak_height)

In [ ]:
#Finding cavity length from peaks

fund_1 = x0s[3]
fund_2 = x0s[4]
print x0s
dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length 
lengths = np.append(lengths,length)

In [ ]:
reload(sa_2D)
sa_2D.peaks_from_2D_data(data_dir,plot_peak_locations=False,minimum_peak_height=minimum_peak_height)

3. determine from the plot above which min_voltage and min_frequency you want to use.
check if diamond and air modes are plottted. 
later you can come back here to plot the overlap of peaks and modes for your best outcome.

In [ ]:
reload(sa_2D)
min_voltage = 4
min_frequency = 420
sa_2D.overlap_peaks_and_modes(data_dir=data_dir,
                               conversion_factor=307.e-9,diamond_thickness=4.232e-6,cavity_length=12.462e-6,
                               minimum_peak_height=minimum_peak_height,plot_fit=False,
                               min_voltage = min_voltage,min_frequency=min_frequency)# min_frequency=450


Now: determine which diamond thicknesses and air lengths you want to check, and find the mean squares.
Plot the mean squares to check if the answer is as expected.
Then go back to the above function to see if indeed this gives a good overlap.

In [ ]:
diamond_thicknesses = np.linspace(3.2e-6,5.2e-6,51)
air_lengths = np.linspace(11.05e-6,16.05e-6,51)
reload(sa_2D)
ms_overlaps = sa_2D.find_best_overlap_peaks_and_modes(data_dir,diamond_thicknesses, air_lengths,
                                                      minimum_peak_height=minimum_peak_height,
                                                      min_voltage = min_voltage,min_frequency=min_frequency)

In [ ]:
ix_min_mean_square_overlap = np.unravel_index(ms_overlaps.argmin(), ms_overlaps.shape)
print 'lowest mean square error (',ms_overlaps[ix_min_mean_square_overlap], ') is found for:'
print 'diamond thickness: ',diamond_thicknesses[ix_min_mean_square_overlap[0]]*1.e6, 'um'
print 'air length: ',air_lengths[ix_min_mean_square_overlap[1]]*1.e6, 'um'
print 'total cavity length:', (diamond_thicknesses[ix_min_mean_square_overlap[0]]+air_lengths[ix_min_mean_square_overlap[1]])*1.e6, 'um'


fig,ax = plt.subplots()
ax.set_xlabel('air length (um)',fontsize =16)
ax.set_ylabel('diamond thickness (um)',fontsize = 16)
ax.set_title('mean square deviation')
extent = [air_lengths[0]*1.e6,air_lengths[-1]*1.e6,diamond_thicknesses[-1]*1.e6,diamond_thicknesses[0]*1.e6]
im = ax.imshow(ms_overlaps, extent=extent, interpolation='None', cmap='afmhot',aspect=2)

plt.colorbar(im)  

fig.savefig(os.path.join(data_dir, 'overlap_mean_square_deviations.png'))


In [ ]:
# you can plot the mean square deviation on a log scale to make things more clear. although here vmax is still useful.

fig,ax = plt.subplots()
ax.set_xlabel('air length (um)',fontsize =16)
ax.set_ylabel('diamond thickness (um)',fontsize = 16)
ax.set_title('log square deviation')

extent = [air_lengths[0]*1.e6,air_lengths[-1]*1.e6,diamond_thicknesses[-1]*1.e6,diamond_thicknesses[0]*1.e6]
im = ax.imshow(np.log(ms_overlaps), extent=extent, interpolation='None', cmap='afmhot',aspect=1.6,vmax=0)

plt.colorbar(im)  

fig.savefig(os.path.join(data_dir, 'overlap_log_mean_square_deviations.png'))


In [ ]:
# If you want, you can plot several of the manual overlaps, for consecutive cavity lenghts

reload(sa_2D)

for i in np.arange(-3,4):
    print i 
    cavity_length = 12.e-6 + i*325.e-9
    
    sa_2D.overlap_peaks_and_modes(data_dir=data_dir,
                               conversion_factor=307.e-9,diamond_thickness=4.1e-6,cavity_length=cavity_length,
                               minimum_peak_height=1000,plot_fit=False,min_frequency=450, min_voltage = 0) 


In [ ]:
# If you want, you can plot several of the manual overlaps, for consecutive diamond thicknesses

reload(sa_2D)

for i in np.arange(-3,4):
    print i 
    diamond_thickness = 4.1e-6 + i*133.e-9
    cavity_length = 12.35e-6 + i*133.e-9 
    
    sa_2D.overlap_peaks_and_modes(data_dir=data_dir,
                               conversion_factor=307.e-9,diamond_thickness=diamond_thickness,cavity_length=cavity_length,
                               minimum_peak_height=1000,plot_fit=False,min_frequency=450, min_voltage = 0) 
    plt.show()

    

### check cavity length for several spectrometer scans on bare cavity

In [ ]:
reload(sa_2D)
reload(sa)

lengths = np.array([])
### pick by hand the peaks that are the fundamental modes

#scan nr 0
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,0],plot_peak_locations=False)
fund_1 = x0s[1]
fund_2 = x0s[-1]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length 
lengths = np.append(lengths,length)

#scan nr 1
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,1],plot_peak_locations=False)
fund_1 = x0s[1]
fund_2 = x0s[-1]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)


#scan nr 2
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,2],plot_peak_locations=False)
fund_1 = x0s[2]
fund_2 = x0s[-1]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = 0#ambiguous  c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)

#scan nr 3
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,3],plot_peak_locations=False)
fund_1 = x0s[0]
fund_2 = x0s[4]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)


#scan nr 4
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,4],plot_peak_locations=False)
fund_1 = x0s[1]
fund_2 = x0s[5]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)


#scan nr 5
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,5],plot_peak_locations=False)
fund_1 = x0s[2]
fund_2 = x0s[-1]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)


#scan nr 6
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,6],plot_peak_locations=False)
fund_1 = x0s[2]
fund_2 = x0s[-1]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = 0 # ambiguous c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)

#scan nr 7
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,7],plot_peak_locations=False)
fund_1 = x0s[1]
fund_2 = x0s[4]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)


#scan nr 8
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,8],plot_peak_locations=False)
fund_1 = x0s[2]
fund_2 = x0s[5]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)



#scan nr 9
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,9],plot_peak_locations=False)
fund_1 = x0s[2]
fund_2 = x0s[5]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length =0 #ambiguous c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)


#scan nr 10
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,10],plot_peak_locations=False)
fund_1 = x0s[1]
fund_2 = x0s[3]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)

#scan nr 11
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,11],plot_peak_locations=False)
fund_1 = x0s[2]
fund_2 = x0s[5]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)

#scan nr 12
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,12],plot_peak_locations=False)
fund_1 = x0s[0]
fund_2 = x0s[2]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)

#scan nr 13
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,13],plot_peak_locations=False)
fund_1 = x0s[1]
fund_2 = x0s[3]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)

#scan nr 14
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,14],plot_peak_locations=False)
fund_1 = x0s[2]
fund_2 = x0s[-2]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)

#scan nr 15
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,15],plot_peak_locations=False)
fund_1 = x0s[1]
fund_2 = x0s[3]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)

#scan nr 16
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,16],plot_peak_locations=False)
fund_1 = x0s[1]
fund_2 = x0s[3]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)

#scan nr 17
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,17],plot_peak_locations=False)
fund_1 = x0s[0]
fund_2 = x0s[2]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)

#scan nr 18
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,18],plot_peak_locations=False)
fund_1 = x0s[1]
fund_2 = x0s[4]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)


#scan nr 19
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,19],minimum_peak_height = 10000,plot_peak_locations=False)
fund_1 = x0s[0]
fund_2 = x0s[3]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)


#scan nr 20
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,20],plot_peak_locations=False)
fund_1 = x0s[0]
fund_2 = x0s[2]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)


#scan nr 21
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,21],minimum_peak_height = 10000,plot_peak_locations=False)
fund_1 = x0s[2]
fund_2 = x0s[5]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)


#scan nr 22
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,22],plot_peak_locations=False)
fund_1 = x0s[1]
fund_2 = x0s[3]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)


#scan nr 23
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,23],minimum_peak_height = 10000,plot_peak_locations=False)
fund_1 = x0s[0]
fund_2 = x0s[3]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)

#scan nr 24
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,24],plot_peak_locations=False)
fund_1 = x0s[0]
fund_2 = x0s[3]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)

#scan nr 25
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,25],minimum_peak_height = 10000,plot_peak_locations=False)
fund_1 = x0s[1]
fund_2 = x0s[4]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)



#scan nr 26
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,26],plot_peak_locations=False)
fund_1 = x0s[0]
fund_2 = x0s[2]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)

#scan nr 27
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,27],minimum_peak_height = 10000,plot_peak_locations=False)
fund_1 = x0s[1]
fund_2 = x0s[3]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)


#scan nr 28
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,28],plot_peak_locations=False)
fund_1 = x0s[1]
fund_2 = x0s[2]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)

#scan nr 29
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,29],minimum_peak_height = 10000,plot_peak_locations=False)
fund_1 = x0s[0]
fund_2 = x0s[3]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)


#scan nr 30
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,30],plot_peak_locations=False)
fund_1 = x0s[1]
fund_2 = x0s[4]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)

#scan nr 31
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,31],minimum_peak_height = 10000,plot_peak_locations=True)
fund_1 = x0s[5]
fund_2 = x0s[-3]

dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
length = c/(2*dnu_fsr)
print length
lengths = np.append(lengths,length)



lengths = lengths*1.e6
fig,ax = plt.subplots()
ax.plot(lengths[:],'o')
xticks = np.linspace(ax.get_xlim()[0],ax.get_xlim()[-1],5)
xticklabels = np.linspace(-2,10,5)
ax.set_xticks(xticks)
ax.set_xticklabels(xticklabels)
ax.set_xlabel('V')
ax.set_ylabel('cavity length (um)')
ax.set_ylim(3,8)


plt.figure()
plt.plot(lengths[(np.where(lengths>0))])




In [ ]:
# you can also plot the data in a different way
reload(sa_2D)
sa_2D.shifted_plot_from_2D_data(data_dir)

In [ ]:
#plotting air modes for a specific configuration
reload(sa_2D)
fig,ax = sa_2D.plot_air_modes(cavity_length=0.25e-6,conversion_factor = 1.25e-6/12)
y0=3.e8/(800e-9)*1.e-12
y1=3.e8/(630e-9)*1.e-12
ax.set_ylim(y0,y1)
ax.set_xlabel('length *1.25')
ax.set_ylabel('freq(THz)')